In [1]:
import pandas as pd
from datetime import datetime
import plotly.express as px
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv('week_5.csv', sep='\t',
                 names=["id", "datetime", "lat", "long", "ordre",'week'], 
                 parse_dates=['datetime'], date_parser=dateparse)

In [18]:
df.shape

(297689, 6)

Timestamp('2015-05-12 00:59:59')

In [7]:
min_lat=df['lat'].min()
max_lat=df['lat'].max()
x=0.001
lat_c=min_lat+x

In [8]:
import datetime, time
def see_long(mil,mal):
    df_tmp=df[(df['lat']<=mal) & (df['lat']>=mil) & (df['id']!='DEL')]
    milo=df_tmp['long'].min()
    malo=df_tmp['long'].max()
    ln=len(np.unique(df_tmp['id']))
    if(ln<3 ):
        df.loc[(df['lat']<=mal) & (df['lat']>=mil),'id']='DEL'
    else:
        df_to_ano=df[(df['lat']<=mal) & (df['lat']>=mil) & (df['long']<=malo) & (df['long']>=milo)]
        mean_long=df_to_ano['long'].mean()
        df.loc[(df['long']>=mean_long+0.1) | (df['long']<=mean_long-0.1), 'id'] = 'DEL'
        
        datemin1 = time.mktime(df_to_ano['datetime'].min().timetuple())
        datemax1 = time.mktime(df_to_ano['datetime'].max().timetuple())
        meandate1 = (datemin1 + datemax1)//2
        mean_to_set=datetime.datetime.fromtimestamp(meandate1)
        df.loc[(df['lat']<=mal) 
               & (df['lat']>=mil) 
               & (df['long']<=malo) 
               & (df['long']>=milo)
               ,['lat','long','datetime']] = df_to_ano['lat'].mean(), mean_long,mean_to_set
        

In [9]:
import numpy as np
while lat_c<max_lat:
    df_int=[]
    while x<0.1:
        df_int=df[(df['lat']<lat_c) & (df['lat']>=min_lat) & (df['id']!='DEL')]
        ln=len(np.unique(df_int['id']))
        if(ln>=3):
            z=0.001
            min_int_lat=min_lat
            max_int_lat=lat_c
            int_lat_c=min_int_lat+z
            while int_lat_c<=max_int_lat:
                see_long(min_int_lat,int_lat_c)
                min_int_lat=int_lat_c
                int_lat_c+=z
            break;
        else:
            x+=x
            lat_c+=x
    if((len(np.unique(df_int['id']))<3) | (x>=0.1)):
        df.loc[(df['lat']<lat_c) & (df['lat']>=min_lat), 'id'] = 'DEL'
#         print(f'del between {min_lat} and {lat_c}')
    x=0.001
    min_lat=lat_c
    lat_c=min_lat+x     

df_clean=df[df['id']!='DEL']

In [10]:
fig = px.scatter(df_clean, x="lat", y="long", marginal_x="box", marginal_y="box", color='id')
# fig.show()

In [15]:
df_clean.shape

(213378, 6)

In [13]:
df.shape

(297689, 6)

In [14]:
df.sort_values('ordre')

,id,datetime,lat,long,ordre,week
0,DEL,2015-05-11 11:41:31,2.344210,48.846147,852216,10
1,DEL,2015-05-11 11:41:32,2.343963,48.847593,852217,10
2,DEL,2015-05-11 11:41:34,2.343957,48.847590,852218,10
3,DEL,2015-05-11 11:41:35,2.343960,48.847598,852219,10
4,DEL,2015-05-11 11:41:37,2.343960,48.847598,852220,10
...,...,...,...,...,...,...
297684,DEL,2015-05-12 00:55:52,-17.445577,14.714418,34522043,10
297685,DEL,2015-05-12 00:55:53,-17.445578,14.714418,34522044,10
297686,DEL,2015-05-12 00:55:55,-17.445582,14.714420,34522045,10
297687,DEL,2015-05-12 00:55:56,-17.445582,14.714420,34522046,10


In [17]:
df.drop(['ordre', 'week'], axis=1, inplace=True)

In [124]:
df.to_csv(f'week_5_anony_lat_long.csv', header=False, index=False,sep='\t')
# df.shape